In [1]:
from utils import *


In [37]:


from LOTlib.Grammar import Grammar
import scipy.stats as st
from LOTlib.TopN import TopN

grammar = Grammar(start='BASE')

#grammar.add_rule('EXPR', '(%s + %s)', ['EXPR', 'EXPR'], 1.0)
grammar.add_rule('EXPR', 'mod', ['EXPR', 'EXPR'], 1.0)
grammar.add_rule('EXPR', 'times', ['EXPR', 'EXPR'], 1.0)
grammar.add_rule('EXPR', 'plus', ['EXPR', 'EXPR'], 1.0)
grammar.add_rule('EXPR', 'div', ['EXPR', 'EXPR'], 1.0)
grammar.add_rule('EXPR', '(-%s)', ['EXPR'], 2.0)
#grammar.add_rule('EXPR', 'float(%s)', ['EXPR'], 2.0)

#grammar.add_rule('EXPR', '(%s * %s)', ['EXPR', 'EXPR'], 1.0)
#grammar.add_rule('EXPR', '(float(%s) / float(%s))', ['EXPR', 'EXPR'], 1.0)
grammar.add_rule('EXPR', 'if_else', ['BOOL', 'EXPR', 'EXPR'], 1.0)
grammar.add_rule('BASE', 'if_else(gt(INDEX,2), %s, %s)', ['EXPR', 'EXPR'], 1.0)
#grammar.add_rule('BASE', 'if_else(eq(if_else(eq(INDEX,1), 2))), %s, %s)', ['EXPR', 'EXPR'], 1.0)
#grammar.add_rule('BASE', 'if_else(eq(INDEX,2), %s, %s)', ['EXPR', 'EXPR'], 1.0)
#grammar.add_rule('BASE', 'if_else(eq(INDEX,1), %s, %s)', ['EXPR', 'EXPR'], 1.0)
#grammar.add_rule('EXPR', 'if_else(eq(INDEX,1), %s, %s)', ['EXPR', 'EXPR'], 1.0)
#grammar.add_rule('EXPR', 'if_else(eq(INDEX,2), %s, %s)', ['EXPR', 'EXPR'], 1.0)
grammar.add_rule('BASE', 'base_case(INDEX,%s,%s,%s)', ['EXPR', 'EXPR', 'EXPR'], 1.0)

grammar.add_rule('BASE', '%s', ['EXPR'], 2.0)


#grammar.add_rule('EXPR', '((%s)(%s))',  ['FUNCDEF', 'FUNC'], 1.0)
#grammar.add_rule('FUNCDEF', 'lambda', ['EXPR'], 1.0, bv_type='EXPR', bv_args=['EXPR'], bv_prefix='F')
#grammar.add_rule('FUNC', 'lambda', ['EXPR'], 1.0, bv_type='EXPR')


grammar.add_rule('EXPR', 'PREV', None, 6.0) 
grammar.add_rule('EXPR', 'PREV_2', None, 6.0) 

grammar.add_rule('EXPR', 'INDEX', None, 6.0) 

grammar.add_rule('BOOL', 'eq', ['EXPR', 'EXPR'], 3.0)
grammar.add_rule('BOOL', 'gt', ['EXPR', 'EXPR'], 3.0)
grammar.add_rule('BOOL', 'or_', ['BOOL', 'BOOL'], 1.0)
grammar.add_rule('BOOL', 'and_', ['BOOL', 'BOOL'], 1.0)
#grammar.add_rule('BOOL', 'rand_n', ["EXPR","EXPR"], 1.0)

grammar.add_rule('BOOL', 'eq(INDEX,1)', None, 1.0)
grammar.add_rule('BOOL', 'eq(INDEX,2)', None, 1.0)

for n in xrange(1,8):
    grammar.add_rule('EXPR', str(n), None, 2.)

from math import log
from LOTlib.Hypotheses.LOTHypothesis import LOTHypothesis

USELESS=-1982324537364124
###################################### 
    
# define a 
class MyHypothesisX(LOTHypothesis):
    def __init__(self, **kwargs):
        LOTHypothesis.__init__(self, grammar=grammar, 
                               display="lambda PREV,PREV_2,INDEX: %s", 
                               **kwargs)

    def __call__(self, *args):
        try:
            # try to do it from the superclass
            return LOTHypothesis.__call__(self, *args)
        except ZeroDivisionError:
            # and if we get an error, return nan
            return float("nan")

    def compute_single_likelihood(self, datum):
        alpha = datum.alpha
        log_p = 0.
        
        f_outputs = []
        diff = 0.
        for i in xrange(len(datum.input)):
            if len(f_outputs) == 0:
                generated = self(USELESS,USELESS, i+1)
            elif len(f_outputs) == 1:
                generated = self(f_outputs[-1],USELESS, i+1)
            else:
                generated = self(f_outputs[-1], f_outputs[-2],i+1)
            f_outputs.append(generated)
            if len(f_outputs) <= len(datum.input):
                t_diff= abs(max(-1000, min(1000, f_outputs[i])) - 
                            datum.input[i])**2
                if log(t_diff+1) > 2.5:
                    diff += t_diff
                if generated == datum.input[i]:
                    log_p += log(alpha) 
                else:
                    log_p += log(1-alpha)
                    
            if len(f_outputs) == len(datum.input):
                n_apart = levenshtein(f_outputs, datum.input)
                log_p += (log(1.-alpha) * n_apart +
                        log(alpha) * (len(datum.input) - n_apart))
                log_p += log(max(1.-alpha, 1./(1. + diff)))

            if abs(generated) >= abs(USELESS)/10:
                log_p += log(1-alpha) 
                

                    
        #if len(f_outputs) > len(datum.input):
           # if datum.input in f_outputs:
              #  log_p += log(1.-alpha) * len(datum.input)
           # else:
               # log_p += log(alpha) * len(datum.input)

        return log_p
        
        




In [35]:
from LOTlib.DataAndObjects import FunctionData

# Now our data takes input x=3 and maps it to 12
# What could the function be?
#data_input = [1,4,9,16,25,36]
#data_input = [1,9,2,8,3,7,4,6]
#data_input = [1,3,7,15,31,63]
#data_input = [1,1,2,3,5,8]
#data_input = [1,1,3,5,9,15,25]
#data_input = [1,1,3,7,17,41,99,239]

#data_input = [10,40,10,4]
#data_input = [1,2,2,4,8,32]
#data_input = [2,3,2,6,2,12]


def f_output(hyp, n=10, initial=USELESS):
    ls = [hyp(USELESS,USELESS, 1)]
    ls.append(hyp(ls[-1],USELESS, 2))
    for i in xrange(0,n):
        ls.append(hyp(ls[-1],ls[-2], i+1))
    return ls[2:]

######################################## 
## Actually run
######################################## 
from LOTlib.Inference.Samplers.MetropolisHastings import MHSampler
from LOTlib.SampleStream import *

def main(data, STEPS=1000, n_chains=1,  a_t=1., t_h=25):
    from LOTlib.Inference.Samplers.MetropolisHastings import MHSampler
    from LOTlib.SampleStream import *

    h0 = MyHypothesisX()
    seen = set()
    tn = TopN(N=t_h)
    
    for _ in xrange(n_chains):

        for h in MHSampler(h0, data, 
                steps=STEPS, acceptance_temperature=a_t):     
            rc = get_rule_counts(grammar, h.value)
            ls = f_output(h)
           # if levenshtein(ls[:len(data_input)], data_input) <=1:
               # print h.value, ls

            if len(ls) >= len(data_input) and str(h) not in seen:
                seen.add(str(h))
                tn.add(h)

    top_hyps = tn.get_all(sorted=True)
    return top_hyps



<ipython-input-35-6295e1c060a6>:30: SyntaxWarning: import * only allowed at module level
  def main(data, STEPS=1000, n_chains=1,  a_t=1., t_h=25):


In [43]:
#data_input = [1,1,3,7,17,41,99,239]
#l = raw_input("Please input a sequence (comma-seperated):\n" )
from math import sin
def rand_list(n, low=1,up=10):
    rl = []
    last = (low + up) / 2
    dist = abs(up - low) / 2
    for _ in xrange(n):
        last = random.randint(last - dist - 1, last + dist + 1)
        dist = 1 + abs(dist + random.randint(-2,2))
        rl.append(str(last))
    return ",".join(rl)
o = "id,X,input,output,out_i,rules\n"
id_n=0
lists = ["5,10,16,23,31,40"]  
lists += ["1,1,2,3"]  + ["1,4,8,13"] + ["2,2,3,5,8"]  + ["4,9,5,10"]
lists += ["1,2,4,8,4,2,1,2,4"]
lists += ["1,2,4,8,4,2,1,2"]
lists += ["1,2,4,8,4,2,1"]
lists += ["1,2,4,8,4,2"]
lists += ["1,2,4,8,4"]
lists += ["8,4,2,1"]
lists += ["1,9,1,8,1,7,1,6"]
lists += [rand_list(4,-1,5) for _ in xrange(10)]
lists += [rand_list(5,-1,1) for _ in xrange(10)]

for l in lists[::-1]:
    print l
    data_str = l.replace(",","_")
    data_input = [int(i) for i in l.split(",")]
    print("Thanks! Please wait for me to read your mind...")
    print
    data = [ FunctionData(input=data_input, 
                          output=10, alpha=1.-10e-5)]

    top_hyps = main(data, STEPS=5000,n_chains=2, 
                    a_t=1.5, t_h=30)#output is length
    #unique_seqs = []
    unique_seqs = {}
    seen = set()
    norm = 0.
    for h in top_hyps:
       # print h.value, h.posterior_score, h.likelihood
        #print f_output(h, n=10, initial=USELESS)
        out = str(f_output(h,n=20,initial=USELESS))
        if str(h) not in seen:
            if out not in unique_seqs:
                unique_seqs[out] = [0.,[]]
            unique_seqs[out][0] += exp(h.posterior_score)
            unique_seqs[out][1].append((h.value, exp(h.posterior_score)))

            #unique_seqs.append((f_output(h,n=10,initial=USELESS), 
                         #       str(h),
                             #   exp(h.posterior_score)))
            norm +=  exp(h.posterior_score)
            seen.add(str(h))
        #print
    print("Were you thinking of one of the following?")
    #unique_seqs = sorted(unique_seqs, key=lambda tup: tup[1])
    sort_seqs = []
    for seq in unique_seqs:
        post = unique_seqs[seq][0] / norm
        hs = [(i[0], i[1]/norm) for i in unique_seqs[seq][1]]
        hs = sorted(hs, key=lambda tup: -tup[1])
        unique_seqs[seq] = (post, hs)
        sort_seqs.append((seq,hs[0][0],post))
        rc = [[str(i) for i in list(l)] for l in 
              get_rule_counts(grammar, hs[0][0])]
        str_rc = ["".join(r) for r in rc]
        str_rc = "_".join(str_rc)
        seq_str = seq.replace("[", "").replace("]","").replace(", ","_")
        seq_str = seq_str.replace(".","p")
        seq_l = seq_str.replace("p0", "").split("_")
        for i in xrange(len(seq_l)):
            o += "%d,%d,%s,%s,%s,%s\n" % (id_n,i,data_str,seq_str,seq_l[i],str_rc)
        id_n += 1
    sort_seqs = sorted(sort_seqs, key=lambda tup: -tup[2])
    for i in sort_seqs: 
        if i[2] > 0.001:
            new_k = []
            str_i = i[0].replace("[", "").replace("]","").replace(", ","_")
            seq_l = str_i.split("_")
            for k in seq_l:
                if abs(int(float(k)) - float(k)) < 0.01:
                    new_k.append(int(float(k)))
                else:
                    new_k.append(float(k))

            print new_k, i[2]

    print


f=open("output/out2.csv","w+")
f.write(o)
f.close()


0,-4,2,7,15
Thanks! Please wait for me to read your mind...

Were you thinking of one of the following?
[0, -4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7] 0.792421746293
[0, -4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 0.207578253707

2,4,3,3,5
Thanks! Please wait for me to read your mind...

Were you thinking of one of the following?
[2, 4, 3, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] 0.76171875
[2, 4, 3, 3, 5, 3, 3, 8, 3, 3, 11, 3, 3, 14, 3, 3, 17, 3, 3, 20] 0.0703125
[2, 4, 3, 3, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6] 0.0703125
[2, 4, 3, 3, 5, 6, 3, 3, 9, 10, 3, 3, 13, 14, 3, 3, 17, 18, 3, 3] 0.046875
[2, 4, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 0.02734375
[2, 4, 3, 3, 5, 3, 3, 5, 3, 3, 5, 3, 3, 5, 3, 3, 5, 3, 3, 5] 0.0234375

-2,-3,-2,2,-1
Thanks! Please wait for me to read your mind...

Were you thinking of one of the following?
[-2, -2, -2, 2, -1, 3, -1, 3, -1, 3, -1, 3, -1, 3, -1, 3, -1, 3, -1, 3] 0.87464059804

Were you thinking of one of the following?
[5, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.577294444192
[5, 3, 2, 1, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5] 0.139864403695
[5, 3, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.138417436411
[5, 3, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.138417436411
[5, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 0.00218410156072
[3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.00163807617054

0,-3,-5,-6
Thanks! Please wait for me to read your mind...

Were you thinking of one of the following?
[0, -2, -5, -3, 0, -2, -5, -3, 0, -2, -5, -3, 0, -2, -5, -3, 0, -2, -5, -3] 0.383136527835
[0, -6, 0, -6, 0, -6, 0, -6, 0, -6, 0, -6, 0, -6, 0, -6, 0, -6, 0, -6] 0.238938355974
[-1, -3, -5, -6, 0, -1, 0, -3, -2, -5, -6, 0, -1, 0, -3, -2, -5, -6, 0, -1] 0.171613183848
[-3, -4, -5, -6, 0, -1, 0, -3, -2, -5, -6, 0, -1, 0, -3, -2, -5, -6, 0, -1] 0.0572043946161
[-5, -5, -6, -6, -6, 0, 0, -5, -5